In [2]:
# %%
from string import Template
import ollama
import pandas as pd

# # Local imports
from market import Market
from portfolio import Portfolio

In [ ]:
# %%
DATASET_FP = "/Users/odai/repos/cs-5588-team-gamestop/datasets/CMIN-US"
STARTING_FUNDS = 10_000  # USD
# LLM_MODEL = "llama3.2:3b"
LLM_MODEL = "qwen2.5:1.5b"

DATE = "2021-01-01"

In [ ]:
# %%
class LanguageModel:
    def __init__(self, model_name, template_fp):
        self.model = model_name
        with open(template_fp, "r") as f:
            self.prompt_template = Template(f.read())

    def get_prompt(
        self,
        date,
        market_json,
        funds,
        portfolio_makeup,
        market_factors,
        world_factors,
        risk_tolerance,
    ):
        return self.prompt_template.substitute(
            date=date,
            market_json=market_json,  # TODO: Add statistical model inputs
            funds=funds,
            portfolio=portfolio_makeup,
            tolerance=risk_tolerance,
            market_factors="",  # TODO: Add market factors
            world_factors="",  # TODO: Add world factors
        )

    def execute_prompt(self, prompt):
        response = ollama.chat(
            model=self.model,
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                },
            ],
        )

        return response["message"]["content"]
    
    def get_stat_model_prediction(self, date):
        path = "/Users/odai/repos/cs-5588-team-gamestop/datasets/stat-model-predictions-2021"
        return pd.read_csv(f"{path}/{date}.csv")

In [3]:
path = "/Users/odai/repos/cs-5588-team-gamestop/datasets/stat-model-predictions-2021"
date = "2021-01-04"

df = pd.read_csv(f"{path}/{date}.csv")
df.head()

,ticker,predicted_price
0,AAPL,132.54
1,BAC,30.39
2,CVX,84.66
3,JNJ,158.67
4,WMT,48.95


In [ ]:
market = Market(DATASET_FP)
portfolio = Portfolio()
portfolio.add_funds(STARTING_FUNDS)

llm = LanguageModel(LLM_MODEL)
prompt = llm.get_prompt(
    DATE,
    market.get_info(date=DATE),
    portfolio.funds,
    portfolio.get_makeup(),
    market_factors=None,
    world_factors=None,
    risk_tolerance="LOW",
)

resp = llm.execute_prompt(prompt)
print(resp)